In [2]:
!pip install python-dotenv

!pip install crewai
!pip install 'crewai[tools]'

!pip install openai
!pip install langchain
!pip install -U langchain-openai
!pip install exa_py

  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
  Using cached langchain_core-0.1.52-py3-none-any.whl.metadata (5.9 kB)
INFO: pip is looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
  Using cached langchain_openai-0.1.8-py3-none-any.whl.metadata (2.5 kB)
  Using cached langchain_openai-0.1.7-py3-none-any.whl.metadata (2.5 kB)
Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)
Using cached langchain_core-0.1.52-py3-none-any.whl (302 kB)
Using cached langchain_openai-0.1.7-py3-none-any.whl (34 kB)
  Attempting uninstall: python-dotenv
    Found existing installation: python-dotenv 0.21.0
    Uninstalling python-dotenv-0.21.0:
      Successfully uninstalled python-dotenv-0.21.0
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.2.9
    Uninstalling langchain-core-0.2.9:
      Successfully uninstalled langchain-core-0.2.9
  Attempting 

In [1]:
import os
from dotenv import load_dotenv

import openai
from langchain_openai import ChatOpenAI

'''
os.environ["SERPER_API_KEY"] 
os.environ["EXA_API_KEY"] 
os.environ["OPENAI_API_KEY"]
'''
load_dotenv()

openai.api_key = os.environ['OPENAI_API_KEY']

In [2]:
from crewai import Agent, Task, Crew, Process
from crewai_tools import tool, SerperDevTool, ScrapeWebsiteTool, EXASearchTool

from exa_py import Exa

# Initialize the tool for internet searching capabilities
search_tool = SerperDevTool()
webscrape_tool = ScrapeWebsiteTool()


@tool("Exa search and get contents")
def search_and_get_contents_tool(question: str) -> str:
    """Tool using Exa's Python SDK to run semantic search and return result highlights."""

    exa = Exa(os.environ['EXA_API_KEY'])

    response = exa.search_and_contents(
        question,
        type="neural",
        use_autoprompt=True,
        num_results=3,
        highlights=True
    )

    parsedResult = ''.join([f'<Title id={idx}>{eachResult.title}</Title><URL id={idx}>{eachResult.url}</URL><Highlight id={idx}>{"".join(eachResult.highlights)}</Highlight>' for (idx, eachResult) in enumerate(response.results)])

    return parsedResult

exa_tools = search_and_get_contents_tool

In [3]:
openAIGPT35_500 = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0,
    max_tokens=500,
    timeout=None,
    max_retries=2,
)

openAIGPT35_3000 = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0,
    max_tokens=3000,
    timeout=None,
    max_retries=2,
)

openAIGPT4 = ChatOpenAI(
    model="gpt-4",
    temperature=0,
    max_tokens=3000,
    timeout=None,
    max_retries=2,
)

In [30]:
manager = Agent(
    role='Manager',
    goal='Manages the distribution of user input and synthesizes expert discussions to create a comprehensive portrayal about a historical figure or influential person.',
    verbose=True,
    backstory=(
        "Highly organized professional with experience in project management and coordination. \
        Holds a degree in a relevant field such as history, sociology, or library science, \
        with additional training in collaborative processes and interdisciplinary research. \
        Skilled in communication and synthesis of diverse information sources."
    ),
    llm = openAIGPT35_3000,
    max_iter = 10,
)

manager_task1 = Task(
    description=(
        "Our team's goal is to research and analyze personality and bibliographical profile about {historical_figure}"
        "so that we can provide an opinion as {historical_figure}; i.e., you are impersonating {historical_figure} and"
        "what potential reponse they would have for {question} considering their info (that will be researched by the team)."
        "Since we are trying to build a person's personality, some examples of important qualities would be"
        "talking or writing habits, personal beliefs, bibliography, time era and trends, etc are important when researching."
        "Plan, divide, and distribute appropriate tasks to subject matter experts by \
        providing specific areas of focus and context for their investigation about {historical_figure} like mentioned."
        "Ensure that each expert understands their task and encourage collaboration and discussion among them. \
        This facilitates a thorough exploration of the historical figure’s personality, context, and impact, \
        leveraging the diverse expertise of the team."
    ),
    expected_output=(
        "Give a list of how task should be distributed among subject experts to analyze and investigate about {historical_figure}."
        "Example output format: "
        "**Goal**"
        "- Our goal is to work together as a team to analyze and investigate about {historical_figure}."
        "***Agent Biographer's Tasks**"
        "- As you are a expert in (what the agent is an expert at and its ), you're goal is to summarize bibliographical information about {historical_figure}."
        "- Task number 1"
        "- ...more tasks for each agents to achieve their goal"
    ),
    agent=manager,
)

In [24]:
biographer = Agent(
    role='Biographer',
    goal='Focuses on compiling a detailed and accurate biography of a historical figure',
    verbose=True,
    memory=True,
    backstory=(
        "You are an accomplished author with a background in journalism or literature. \
        Renowned for writing several critically acclaimed biographies, \
        demonstrating expertise in storytelling and interviewing techniques."
    ),
    llm = openAIGPT35_500,
    max_iter = 10,
)

biographer_task_desc = "Gather comprehensive information about the life of {historical_figure}. \
    Conduct interviews with people who knew the {historical_figure}. \
    Write a narrative that captures the experiences, achievements, and personality of {historical_figure}. \
    Additionally when using tools, make sure to not read sources that exceed more than 1000 words \
    and NEVER EVER DO NOT source from https://www.wikipedia.org and any of its sub-domains."

# Research task
biographer_task = Task(
    description = biographer_task_desc,
    expected_output="A comprehensive 3 paragraphs long report on the historical background of the person/subject in research",
    tools=[search_tool, webscrape_tool],
    agent=biographer,
    async_execution = True,
    context = [manager_task1]
)

In [25]:
psychologist = Agent(
    role='Psychologist',
    goal='Analyzes the personality, behavior, and psychological traits of a historical figure or someone influential',
    verbose=True,
    backstory=(
        "Holds a PhD in Psychology or Psychohistory. \
        Experienced clinical psychologist or researcher with a robust publication record in psychological journals, \
        specializing in history and psychological profiles."
    ),
    llm = openAIGPT35_500,
    max_iter = 10, 
)

psychologist_task_desc = "Apply psychological theories to understand the actions and decisions of the {historical_figure}. \ \
    Use psychohistorical methods to interpret the impact of psychological factors on historical events. \
    Provide insights into the mental and emotional state of the person/subject that is being researched. \
    Additionally when using tools, make sure to not read sources that exceed more than 1000 words \
    and NEVER EVER DO NOT source from https://www.wikipedia.org and any of its sub-domains."

# Research task
psychologist_task = Task(
    description = psychologist_task_desc, 
    expected_output='A comprehensive 3 paragraphs long report on the psychological analysis of the {historical_figure} given by manager.',
    tools=[exa_tools, search_tool, webscrape_tool],
    agent=psychologist,
    async_execution = True,
    context = [manager_task1]
)

In [32]:
philosopher = Agent(
    role='Philosopher',
    goal='Analyzes the ethical and philosophical beliefs and contributions made by a historical figure.',
    verbose=True,
    backstory=(
        "PhD in Philosophy or Ethics, with a focus on ethical analysis and philosophical inquiry. \
        Published numerous scholarly articles and books on ethical and philosophical topics, \
        providing deep insights into moral considerations. "
    ),
    llm = openAIGPT35_500,
    max_iter = 10,
)

philosopher_task_desc = "Analyze moral and ethical implications of the decisions made by {historical_figure}. \
    Study philosophical influences on the beliefs and actions made by {historical_figure}. \
    Provide insights into the ethical context of the time period when {historical_figure} lived. \
    Additionally when using tools, make sure to not read sources that exceed more than 1000 words \
    and NEVER EVER DO NOT source from https://www.wikipedia.org and any of its sub-domains."

# Research task
philosopher_task = Task(
    description = philosopher_task_desc,
    expected_output='A comprehensive 3 paragraphs long report and a summarized bullet points of key philosophical ideas of {historical_figure}',
    tools=[exa_tools, search_tool, webscrape_tool],
    agent=philosopher,
    async_execution = True,
    context = [manager_task1]
)

In [33]:
linguist = Agent(
    role='Linguist',
    goal='Analyzes the language, writings, and speeches of historical figures.',
    verbose=True,
    backstory=(
        "PhD in Linguistics, with extensive research on language patterns and communication styles. \
        Published numerous papers on linguistics and conducted significant fieldwork or research."
    ),
    llm = openAIGPT35_500,
)

linguist_task_desc = "Analyzes writings, speeches, linguistic patterns and other communications by the {historical_figure}. \
    Provide context on how language shaped and was shaped by the culture of the historical figure being discussed. \
    Additionally when using tools, make sure to not read sources that exceed more than 1000 words \
    and NEVER EVER DO NOT source from https://www.wikipedia.org and any of its sub-domains."

# Research task
linguist_task = Task(
    description = linguist_task_desc,
    expected_output='A comprehensive 3 paragraphs long report on the psychological analysis of {historical_figure}',
    tools=[exa_tools, search_tool, webscrape_tool],
    agent=linguist,
    async_execution = True,
    context = [manager_task1]
)

In [52]:
manager_task2 = Task(
    description = (
        "Using previous discussion between teammates and what they researched about {historical_figure},"
        "You have to provide a user experience where the user feels like they are talking directly to {historical_figure}."
        
        "To do so, ask and discuss with the experts / team members about what, according to their"
        "profession and knowledge what they think {historical_figure}'s response would be about the question '{question}'."
        
        "Then, come up with a single conclusion of what {historical_figure}'s response would be and "
        "write a single response while acting/role-playing as {historical_figure} character. "
        
        "An example discussion between the team members would look something like below:"
        "*** Historical Figure: {historical_figure} ***"
        "*** Question: Is capitalism good? ***"
        "*Biographer's opinion*"
        "I think {historical_figure} would answer to the question 'is capitalism good' by '...' because according to"
        "what I have researched earlier '...'"
        "*Philosopher's opinion*"
        "I think {historical_figure} would answer to the question 'is capitalism good' by '...' because according to"
        "his philosophical beliefs from his work '...'"
        "..."
    ),
    expected_input = (
        "Give a list of how task should be distributed among subject experts to analyze and investigate about {historical_figure}."
        "Example output format: "
        "**Goal**"
        "- Our goal is to work together as a team to analyze and investigate about {historical_figure}."
        "***Agent Biographer's Tasks**"
        "- As you are a expert in (what the agent is an expert at and its ), you're goal is to summarize bibliographical information about {historical_figure}."
        "- Task number 1"
        "- ...more tasks for each agents to achieve their goal"
    ),
    expected_output = (
        "A single summarized response of what the team, as a whole after discussion with each other, think"
        "{historical_figure} would have responded to the question '{question}'."
        "The response must clearly state and show the logical steps and reasons why you think that way about {question} while"
        "also making sure that the final, single summarized response is written like an impersonation of {historical_figure} is reading out the response."
    ),
    agent=manager,
    context = [manager_task1, biographer_task, psychologist_task, philosopher_task, linguist_task],
    async_execution = True,
    human_input = True  
)

In [53]:
# Instantiate your crew with a sequential process
crew = Crew(
    agents=[manager, biographer, psychologist, linguist, philosopher],
    tasks=[
        manager_task1, biographer_task, psychologist_task, philosopher_task, linguist_task, manager_task2
    ],
    # verbose=2, # You can set it to 1 or 2 to different logging levels
)

# Get your crew to work! 
result = crew.kickoff(inputs = {
    "historical_figure": 'Plato',
    "question": "What do you think of capitalism?",
})

2024-06-23 12:56:38,510 - 8475823104 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed




> Entering new CrewAgentExecutor chain...
I need to effectively delegate tasks to the subject matter experts in order to thoroughly research and analyze Plato's personality and bibliographical profile. I must provide clear context and guidance to each expert to ensure they understand their role in the investigation.

Action:
Delegate work to coworker
Action Input:
{
    "coworker": "Biographer",
    "task": "Summarize bibliographical information about Plato",
    "context": "Plato is a renowned philosopher from ancient Greece, known for his works such as 'The Republic' and 'Symposium'. The biographer should focus on gathering and summarizing key information about Plato's life, works, and impact on philosophy."
}

> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:
Plato, born around 428 BC in Athens, Greece, was a philosopher and mathematician in ancient Greece. He was a student of Socrates and the teacher of Aristotle. Plato founded the Academy in 

2024-06-23 12:56:59,721 - 6398439424 - __init__.py-__init__:369 - ERROR: Exception while exporting Span batch.
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniconda/base/envs/hackathon/lib/python3.10/site-packages/urllib3/connectionpool.py", line 466, in _make_request
    self._validate_conn(conn)
  File "/opt/homebrew/Caskroom/miniconda/base/envs/hackathon/lib/python3.10/site-packages/urllib3/connectionpool.py", line 1095, in _validate_conn
    conn.connect()
  File "/opt/homebrew/Caskroom/miniconda/base/envs/hackathon/lib/python3.10/site-packages/urllib3/connection.py", line 652, in connect
    sock_and_verified = _ssl_wrap_socket_and_match_hostname(
  File "/opt/homebrew/Caskroom/miniconda/base/envs/hackathon/lib/python3.10/site-packages/urllib3/connection.py", line 805, in _ssl_wrap_socket_and_match_hostname
    ssl_sock = ssl_wrap_socket(
  File "/opt/homebrew/Caskroom/miniconda/base/envs/hackathon/lib/python3.10/site-packages/urllib3/util/ssl_.py", line 465,

I now can give a great answer

Final Answer:
Plato, the renowned ancient Greek philosopher, was born in Athens in 427 BC to an aristocratic family. His early life was greatly influenced by his father, Ariston, who traced his lineage back to the legendary Greek lawmaker Solon. Plato's mother, Perictione, was related to the famous Greek statesman Critias. 

Plato received a well-rounded education in Athens, studying under the guidance of some of the most prominent thinkers of his time, including Socrates, whose philosophical teachings would have a profound impact on his own work. He also studied mathematics, astronomy, and gymnastics, as was customary for young Athenian men of his social class.

The execution of Socrates in 399 BC had a lasting influence on Plato, leading him to question the nature of justice, truth, and the ideal society. This event played a significant role in shaping his philosophical beliefs and would later be reflected in his famous dialogues, such as "The Republic"

Exception in thread Thread-199 (_execute):
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniconda/base/envs/hackathon/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/opt/homebrew/Caskroom/miniconda/base/envs/hackathon/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/homebrew/Caskroom/miniconda/base/envs/hackathon/lib/python3.10/site-packages/crewai/task.py", line 196, in _execute
    result = agent.execute_task(
  File "/opt/homebrew/Caskroom/miniconda/base/envs/hackathon/lib/python3.10/site-packages/crewai/agent.py", line 243, in execute_task
    result = self.agent_executor.invoke(
  File "/opt/homebrew/Caskroom/miniconda/base/envs/hackathon/lib/python3.10/site-packages/langchain/chains/base.py", line 163, in invoke
    raise e
  File "/opt/homebrew/Caskroom/miniconda/base/envs/hackathon/lib/python3.10/site-packages/langchain/chains/base.py", line 153, in invoke
    self

...

Exception in thread Thread-198 (_execute):
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniconda/base/envs/hackathon/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/opt/homebrew/Caskroom/miniconda/base/envs/hackathon/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/homebrew/Caskroom/miniconda/base/envs/hackathon/lib/python3.10/site-packages/crewai/task.py", line 196, in _execute
    result = agent.execute_task(
  File "/opt/homebrew/Caskroom/miniconda/base/envs/hackathon/lib/python3.10/site-packages/crewai/agent.py", line 243, in execute_task
    result = self.agent_executor.invoke(
  File "/opt/homebrew/Caskroom/miniconda/base/envs/hackathon/lib/python3.10/site-packages/langchain/chains/base.py", line 163, in invoke
    raise e
  File "/opt/homebrew/Caskroom/miniconda/base/envs/hackathon/lib/python3.10/site-packages/langchain/chains/base.py", line 153, in invoke
    self

Thought: I need to continue analyzing Plato's writings, speeches, linguistic patterns, and communication styles to understand the cultural context that shaped his language and how he, in turn, influenced the culture of his time. I should focus on his use of language, rhetoric, and any linguistic innovations he introduced in his works.

Action: Delegate work to coworker
Action Input:
{
    "coworker": "Biographer",
    "task": "Summarize key works written by Plato, such as 'The Republic' and 'Symposium.'"
}

> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:
Plato, one of the most influential philosophers in history, wrote several key works that have had a lasting impact on Western thought. Among his most famous works are "The Republic" and "Symposium." In "The Republic," Plato explores the nature of justice, the ideal state, and the role of the philosopher-king. This work also delves into the concept of the divided line and the allegory of the cave, 

2024-06-23 12:57:29,746 - 6398439424 - __init__.py-__init__:369 - ERROR: Exception while exporting Span batch.
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniconda/base/envs/hackathon/lib/python3.10/site-packages/urllib3/connectionpool.py", line 466, in _make_request
    self._validate_conn(conn)
  File "/opt/homebrew/Caskroom/miniconda/base/envs/hackathon/lib/python3.10/site-packages/urllib3/connectionpool.py", line 1095, in _validate_conn
    conn.connect()
  File "/opt/homebrew/Caskroom/miniconda/base/envs/hackathon/lib/python3.10/site-packages/urllib3/connection.py", line 652, in connect
    sock_and_verified = _ssl_wrap_socket_and_match_hostname(
  File "/opt/homebrew/Caskroom/miniconda/base/envs/hackathon/lib/python3.10/site-packages/urllib3/connection.py", line 805, in _ssl_wrap_socket_and_match_hostname
    ssl_sock = ssl_wrap_socket(
  File "/opt/homebrew/Caskroom/miniconda/base/envs/hackathon/lib/python3.10/site-packages/urllib3/util/ssl_.py", line 465,

Thought: I now can give a great answer
Final Answer: Plato, the renowned ancient Greek philosopher, was born in Athens in 427 BC to an aristocratic family. His early life was greatly influenced by his father, Ariston, and his mother, Perictione, who were both connected to prominent figures in Greek society. Plato received a comprehensive education in Athens, studying under the guidance of notable thinkers like Socrates, whose teachings profoundly impacted his philosophical work. The execution of Socrates in 399 BC played a pivotal role in shaping Plato's philosophical beliefs, leading him to explore themes of justice, truth, and the ideal society in his famous dialogues, such as "The Republic" and "The Apology."

Plato's philosophical ideas continue to hold enduring relevance in contemporary society, particularly his theory of forms and the tripartite soul. His emphasis on seeking knowledge, cultivating reason, and questioning societal norms resonates with modern discussions on ethics,

Exception in thread Thread-206 (_execute):
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniconda/base/envs/hackathon/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/opt/homebrew/Caskroom/miniconda/base/envs/hackathon/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/homebrew/Caskroom/miniconda/base/envs/hackathon/lib/python3.10/site-packages/crewai/task.py", line 196, in _execute
    result = agent.execute_task(
  File "/opt/homebrew/Caskroom/miniconda/base/envs/hackathon/lib/python3.10/site-packages/crewai/agent.py", line 243, in execute_task
    result = self.agent_executor.invoke(
  File "/opt/homebrew/Caskroom/miniconda/base/envs/hackathon/lib/python3.10/site-packages/langchain/chains/base.py", line 163, in invoke
    raise e
  File "/opt/homebrew/Caskroom/miniconda/base/envs/hackathon/lib/python3.10/site-packages/langchain/chains/base.py", line 153, in invoke
    self